This finds all of the account numbers that get used, but don't have an account name mapped to it. Notably, each "rougue" account seems to either only get used once, or only gets used a few times.

In [1]:
import pandas as pd
import csv
import warnings
warnings.filterwarnings('ignore') # Disable warnings for now

In [2]:
df_budget = pd.read_csv("../data/Expense_Budget.csv")

In [3]:
# Map of account # to account name
acct_map = dict((int(row[0]), row[1]) for row in csv.reader(open("../data/account_map.csv", "r")))

# Map of service # to service name
srvc_map = dict((int(row[0]), row[1]) for row in csv.reader(open("../data/service_map.csv", "r")))

In [4]:
df_trans = pd.read_csv("../data/Trans2.csv")

# Convert the dates from strings to datetimes
# TODO: Figure out how to do this faster and without Python crying
df_trans["Date"] = pd.to_datetime(df_trans["Date"])

In [8]:
year = 2013

while year < 2018:
    print("[%d]" % year)

    # Operate only on the budget for the given year and the related subset of transactions
    df_budget_year = df_budget[df_budget.Year == year]    
    df_trans_year = df_trans[df_trans["Date"].dt.year == year]
    
    for dept_name in sorted(df_budget_year["Dept Name"].unique()):
        df_budget_dept = df_budget_year[df_budget_year["Dept Name"] == dept_name]
        fund_num = df_budget_dept["Fund #"].unique()[0]

        for service_num in sorted(df_budget_dept["Service #"].unique()):
            entity = str(fund_num) + "-" + str(service_num)

            # Get all of the transactions under the generated entity
            df_trans_entity = df_trans_year[df_trans_year["Entity"] == entity]
            
            missing_acct_nums = []
            
            for acct_num in sorted(df_trans_entity["Account"].unique()):
                df_trans_acct = df_trans_entity[df_trans_entity["Account"] == acct_num]

                if acct_num not in acct_map:
                    missing_acct_nums.append(acct_num)

            if missing_acct_nums:
                # dept_names get printed multiple times, but fixing it's
                # more trouble than it's worth
                print("\t%s" % dept_name)
                print("\t\t%s [%s]" % (srvc_map[service_num], entity))

                for acct_num in missing_acct_nums:
                    df_trans_acct = df_trans_entity[df_trans_entity["Account"] == acct_num]
                    print("\t\t\tUnknown Account [%s] - %f (%f)" % (acct_num, df_trans_acct["Amount"].sum(), df_trans_acct["Amount"].mean()))

    year += 1

[2013]
	Emergency Services
		Medical-Emergency Med Services [100-320410]
			Unknown Account [55360] - 258000.000000 (64500.000000)
	Fleet Operation
		Fleet Operation [100-191110]
			Unknown Account [55360] - 8375.000000 (8375.000000)
	Health Care & Wellness
		Medical - Choice Plus [200-197135]
			Unknown Account [49440] - -299166.570000 (-299166.570000)
	Law Enforcement
		SO-S Emer Response Team-FF DOJ [100-310234]
			Unknown Account [55010] - 311096.070000 (103698.690000)
			Unknown Account [55100] - 1807.170000 (1807.170000)
	Mental Health
		Area Administration [100-220002]
			Unknown Account [45310] - 0.000000 (0.000000)
	Mental Health
		AMH - Inpatient [100-220225]
			Unknown Account [44628] - -201750.000000 (-201750.000000)
	Public Health
		Human Resrcs-Fiscal-Technology [100-210002]
			Unknown Account [45310] - 0.000000 (0.000000)
			Unknown Account [55360] - 86811.200000 (43405.600000)
	Soil & Water Conservation
		Soil & Water Conservation [100-420110]
			Unknown Account [44112]